# Temporal aggregation

This is a demonstration of how to run temporal aggregations using the cython library code. (Or other aggregations of multiple files into one - not specifically temporal).

The code is written in cython (`raster_utilities/aggregation/temporal/core/temporal.pyx`) and a helper class `raster_utilities/aggregation/temporal/temporal_aggregation_runner.py` is provided to assist with loading the data and passing it to the core function.

This notebook demonstrates how to use the helper code.

Import the aggregation helper class

In [1]:
from raster_utilities.aggregation.temporal.temporal_aggregation_runner import TemporalAggregator


We initialise the class with a dictionary in which the keys are the required output tags (e.g. years) and the values are a list of files corresponding to that period. So, build that object

In [2]:
import glob

In [3]:
inFilePattern = r'G:\modis\mod11a2_v6\Night\*.tif'
#inFilePattern = r'C:\Temp\dataprep\brazil\1km_daily\evi\*_Data.tif'
inFiles = glob.glob(inFilePattern)

Build the dictionary based on extracting the date from the filenames - this will need changing to suit the filename patterns being used and the type of outputs we want (annual, monthly, synoptic months?) 

We can use a defaultdict rather than a real dict which simplifies the loop a bit.

The dictionary key will be used to create the output filenames so you might want to alter the strings slightly to make them more informative.

In [4]:
import os
from collections import defaultdict

In [5]:
daymonths = {1:1, 9:1, 17:1, 25:1, 33:2, 41:2, 49:2, 57:2, 65:3, 73:3, 81:3, 89:3, 97:4, 
             105:4, 113:4, 121:4, 129:5, 137:5, 145:5, 153:6, 161:6, 169:6, 177:6, 185:7, 
             193:7, 201:7, 209:7, 217:8, 225:8, 233:8, 241:8, 249:9, 257:9, 265:9, 273:9, 
             281:10, 289:10, 297:10, 305:10, 313:11, 321:11, 329:11, 337:12, 345:12, 353:12, 
             361:12}

processingKey = defaultdict(list)
# build a dictionary keyed by year, to create annual outputs
tag = "LST_Night_Unfilled."
for fn in inFiles:
    parts = os.path.basename(fn).split('_')
    poo = parts[0]
    yr = poo[1:5]
    daynum = int(poo[5:8])
    poo1 = yr+"."+str(daymonths[daynum]).zfill(2)
    # outkey = tag + poo1
    # processingKey[outkey].append(fn)
    processingKey[tag + str(yr)].append(fn)
    processingKey[tag + str(daymonths[daynum]).zfill(2)].append(fn)

In [6]:
processingKey = defaultdict(list)
for fn in inFiles:
    parts = os.path.basename(fn).split('.')
    yr = parts[1]
    outkey = "CHIRPS."+yr
    processingKey[outkey].append(fn)
    

We also need to specify the output folder, the output nodata value, and whether we want to create a synoptic (overall) output too (this doubles memory use so don't do unless you need it).

In [6]:
outDir = r"G:\modis\mod11a2_v6\Night_Synoptic_Corrected"
outNDV = -9999
doSynoptic = True

Finally we need to specify which stats to do, what's appropriate will depend on the data. For rainfall we just want a sum.
The values must be specified as a list of values from the TemporalAggregationStats class. You can also use TemporalAggregationStats.ALL.

In [7]:
from raster_utilities.aggregation.aggregation_values import TemporalAggregationStats

In [8]:
stats = [ 'mean', 'SD', 'count']

Now we just need to instantiate the class and run the aggregation

In [9]:
agg = TemporalAggregator(processingKey, outDir, outNDV, stats, doSynoptic)

In [10]:
agg.RunAggregation()

Running by splitting across 2 tiles


raster_utilities\aggregation\temporal\temporal_aggregation_runner.py:54: RuntimeWarning: overflow encountered in long_scalars
  bTot = bppTot * nPix


LST_Night_Unfilled.2014
LST_Night_Unfilled.2015
LST_Night_Unfilled.2016
LST_Night_Unfilled.2017
LST_Night_Unfilled.2010
LST_Night_Unfilled.2011
LST_Night_Unfilled.2012
LST_Night_Unfilled.2013
LST_Night_Unfilled.03


raster_utilities\aggregation\temporal\temporal_aggregation_runner.py:56: RuntimeWarning: overflow encountered in long_scalars
  bTot *= 2
raster_utilities\aggregation\temporal\temporal_aggregation_runner.py:57: RuntimeWarning: overflow encountered in long_scalars
  bTot += 8 * nPix # the input data tile


ValueError: Unknown conversion specifier S

Current status as of 27/2:
Night ones: data are correct except synoptic count is doubled
Day ones: all synoptic have been double counted so SDs need adjusting or repeating

In [11]:
agg.doSynoptic

True

In [12]:
[print(k) for f in agg.filesDict.iteritems()]

SyntaxError: invalid syntax (<ipython-input-12-517e84dba13b>, line 1)

In [24]:
n = 0
s = set()
for k,f in agg.filesDict.iteritems():
    print k
    n += len(f)
    s.update(f)
print n

LST_Night_Unfilled.2014
LST_Night_Unfilled.2015
LST_Night_Unfilled.2016
LST_Night_Unfilled.2017
LST_Night_Unfilled.2010
LST_Night_Unfilled.2011
LST_Night_Unfilled.2012
LST_Night_Unfilled.2013
LST_Night_Unfilled.03
LST_Night_Unfilled.02
LST_Night_Unfilled.01
LST_Night_Unfilled.07
LST_Night_Unfilled.06
LST_Night_Unfilled.05
LST_Night_Unfilled.04
LST_Night_Unfilled.09
LST_Night_Unfilled.08
LST_Night_Unfilled.2007
LST_Night_Unfilled.2006
LST_Night_Unfilled.2005
LST_Night_Unfilled.2004
LST_Night_Unfilled.2003
LST_Night_Unfilled.2002
LST_Night_Unfilled.2001
LST_Night_Unfilled.2000
LST_Night_Unfilled.2009
LST_Night_Unfilled.2008
LST_Night_Unfilled.10
LST_Night_Unfilled.11
LST_Night_Unfilled.12
1642


In [26]:
len(s)

821

In [ ]:
s.update()

In [28]:
fOld = agg.filesDict

In [18]:
fNew = {k:v for k,v in fOld.iteritems() if k == 'LST_Night_Unfilled.2013'}

In [19]:
fNew

{'LST_Night_Unfilled.2013': ['G:\\modis\\mod11a2_v6\\Night\\A2013001_LST_Night.tif',
  'G:\\modis\\mod11a2_v6\\Night\\A2013009_LST_Night.tif',
  'G:\\modis\\mod11a2_v6\\Night\\A2013017_LST_Night.tif',
  'G:\\modis\\mod11a2_v6\\Night\\A2013025_LST_Night.tif',
  'G:\\modis\\mod11a2_v6\\Night\\A2013033_LST_Night.tif',
  'G:\\modis\\mod11a2_v6\\Night\\A2013041_LST_Night.tif',
  'G:\\modis\\mod11a2_v6\\Night\\A2013049_LST_Night.tif',
  'G:\\modis\\mod11a2_v6\\Night\\A2013057_LST_Night.tif',
  'G:\\modis\\mod11a2_v6\\Night\\A2013065_LST_Night.tif',
  'G:\\modis\\mod11a2_v6\\Night\\A2013073_LST_Night.tif',
  'G:\\modis\\mod11a2_v6\\Night\\A2013081_LST_Night.tif',
  'G:\\modis\\mod11a2_v6\\Night\\A2013089_LST_Night.tif',
  'G:\\modis\\mod11a2_v6\\Night\\A2013097_LST_Night.tif',
  'G:\\modis\\mod11a2_v6\\Night\\A2013105_LST_Night.tif',
  'G:\\modis\\mod11a2_v6\\Night\\A2013113_LST_Night.tif',
  'G:\\modis\\mod11a2_v6\\Night\\A2013121_LST_Night.tif',
  'G:\\modis\\mod11a2_v6\\Night\\A2013129_LST

In [29]:
agg.synopticFileSet

set()